# Scraping Magalu

### Pré-requisitos
- Selenium
- BeautifulSoup
- Pandas
- Unidecode

In [33]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from unidecode import unidecode

### Função para pesquisar e personalizar a URL

In [3]:
def personalizar_url(termo_de_pesquisa):
    """
    Função para pesquisar produtos e personalizar a url de pesquisa
    IN: 'smart tv 4k': str
    OUT: https://www.magazineluiza.com.br/busca/iphone 11/?page={}
    """
    template = 'https://www.magazineluiza.com.br/busca/{}/'
    termo_de_pesquisa.replace('', '+')
    
    # Adicionando o query string da pesquisa na URL
    url = template.format(termo_de_pesquisa)
    
    # Adicionando o query string da página na URL
    url += '?page={}'
    
    return url

### Scraping Smartphones

In [43]:
url = personalizar_url('smartphone').format(1)
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

### Extração da Lista de Produtos

In [15]:
lista_de_produtos = soup.find_all('li', {'class':'sc-eCVOVf loRbcV'})
len(lista_de_produtos)

59

### Prototipando a extração de um registro

In [17]:
item = lista_de_produtos[0]

### Extração do Título do Produto

In [21]:
titulo_produto = item.a.h2.text.strip()
titulo_produto

'iPhone 11 Apple 64GB Branco 6,1” 12MP iOS'

### Extração URL dos Detalhes do Produto

In [24]:
rota_detalhes_produto = item.a.get('href')
rota_detalhes_produto

'/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/'

In [25]:
url_detalhes_produto = f'https://www.magazineluiza.com.br{rota_detalhes_produto}'
url_detalhes_produto

'https://www.magazineluiza.com.br/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/'

### Extração do Preço

In [30]:
preco_produto = item.find('p', {'data-testid':'price-value'}).text.strip()
preco_produto

'R$\xa03.719,07'

#### Problema com \xa0.
Resolução:
https://stackoverflow.com/questions/26068832/how-to-remove-this-xa0-from-a-string-in-python

In [34]:
preco_produto = item.find('p', {'data-testid':'price-value'}).text.strip()
preco_produto = unidecode(preco_produto)
preco_produto

'R$ 3.719,07'

#### Extração Parte Inteira do Preço

In [42]:
preco_produto_float = float(preco_produto.split()[-1].replace('.','').replace(',','.'))
preco_produto_float

3719.07

### Extraindo Avaliação

Conta-se a avaliação na magalu ao contar o número de estrelas completas -> StarIcon

In [46]:
avaliacao = len(item.find_all('use', {'xlink:href':'#StarIcon'}))
avaliacao

4

### Extraindo Quantidade de Avaliações

In [59]:
quantidade_avaliacoes = int(item.find('span', {'format':'count'}).text.strip())
quantidade_avaliacoes

58

## Encapsulando o Protótipo em Funções
Isto torna nosso código mais limpo e reutilizável

In [69]:
def extrair_titulo_produto(produto):
    return produto.a.h2.text.strip()

def extrair_url_produto(produto):
    rota_detalhes_produto = produto.a.get('href')
    return f'https://www.magazineluiza.com.br{rota_detalhes_produto}'

def extrair_preco_produto(produto):
    preco_produto = produto.find('p', {'data-testid':'price-value'}).text.strip()
    preco_produto = unidecode(preco_produto)
    preco_produto_float = float(preco_produto.split()[-1].replace('.','').replace(',','.'))
    return preco_produto_float
    
def extrair_avaliacao(produto):
    return len(produto.find_all('use', {'xlink:href':'#StarIcon'})) 

def extrair_quantidade_avaliacoes(produto):
    return int(produto.find('span', {'format':'count'}).text.strip())

### Agregando os códigos para a função de extrair informações do produto

In [64]:
def extrair_informacoes_produto(produto):
    titulo = extrair_titulo_produto(produto)
    preco = extrair_preco_produto(produto)
    avaliacao = extrair_avaliacao(produto)
    quantidade_avaliacoes = extrair_quantidade_avaliacoes(produto)
    url = extrair_url_produto(produto)
    
    return (titulo, preco, avaliacao, quantidade_avaliacoes, url)

#### Testando a função

In [65]:
print(extrair_informacoes_produto(item))

('iPhone 11 Apple 64GB Branco 6,1” 12MP iOS', 3719.07, 4, 58, 'https://www.magazineluiza.com.br/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/')


Deu tudo certo até agora 😁

### Testando para saber se pega todos os itens da página com sucesso

In [67]:
lista_produtos = []
resultado_da_pesquisa = soup.find_all('li', {'class':'sc-eCVOVf loRbcV'})

for produto in resultado_da_pesquisa:
    lista_produtos.append(extrair_informacoes_produto(produto))

AttributeError: 'NoneType' object has no attribute 'text'

Deu errado ao extrair a quantidade de avaliações 😅

Podemos corrigir isso através do tratamento de exceções

In [70]:
def extrair_titulo_produto(produto):
    return produto.a.h2.text.strip()

def extrair_url_produto(produto):
    rota_detalhes_produto = produto.a.get('href')
    return f'https://www.magazineluiza.com.br{rota_detalhes_produto}'

def extrair_preco_produto(produto):
    preco_produto = produto.find('p', {'data-testid':'price-value'}).text.strip()
    preco_produto = unidecode(preco_produto)
    preco_produto_float = float(preco_produto.split()[-1].replace('.','').replace(',','.'))
    return preco_produto_float
    
def extrair_avaliacao(produto):
    return len(produto.find_all('use', {'xlink:href':'#StarIcon'})) 

def extrair_quantidade_avaliacoes(produto):
    try:
        return int(produto.find('span', {'format':'count'}).text.strip())
    except AttributeError:
        return 0

Testando novamente...

In [71]:
lista_produtos = []
resultado_da_pesquisa = soup.find_all('li', {'class':'sc-eCVOVf loRbcV'})

for produto in resultado_da_pesquisa:
    lista_produtos.append(extrair_informacoes_produto(produto))

In [72]:
print(lista_produtos)

[('iPhone 11 Apple 64GB Branco 6,1” 12MP iOS', 3719.07, 4, 58, 'https://www.magazineluiza.com.br/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/'), ('iPhone 11 Apple 64GB Roxo 6,1” 12MP iOS', 3719.07, 5, 1, 'https://www.magazineluiza.com.br/iphone-11-apple-64gb-roxo-61-12mp-ios/p/155610900/te/ip11/'), ('Smartphone Samsung Galaxy S20 FE 128GB Cloud Mint', 2099.0, 4, 48, 'https://www.magazineluiza.com.br/smartphone-samsung-galaxy-s20-fe-128gb-cloud-mint-4g-6gb-ram-tela-65-cam-tripla-selfie-32mp/p/155630400/te/gs2f/'), ('iPhone 11 Apple 64GB Preto 6,1” 12MP iOS', 3719.07, 4, 80, 'https://www.magazineluiza.com.br/iphone-11-apple-64gb-preto-61-12mp-ios/p/155610500/te/ip11/'), ('Smartphone Samsung Galaxy S20 FE 128GB Cloud White', 2099.0, 4, 51, 'https://www.magazineluiza.com.br/smartphone-samsung-galaxy-s20-fe-128gb-cloud-white-4g-6gb-ram-tela-65-cam-tripla-selfie-32mp/p/155630000/te/gs2f/'), ('iPhone XR Apple 64GB Preto 6,1” 12MP iOS', 2999.0, 4, 88, 'https://www.magazineluiza

Agora deu tudo certo! 😎

### Encapsulamento da Função de Extrair Produtos da Página

In [75]:
def extrair_produtos_pagina():
    resultado_da_pesquisa = soup.find_all('li', {'class':'sc-eCVOVf loRbcV'})
    return [extrair_informacoes_produto(produto) for produto in resultado_da_pesquisa]

Testando a função encapsulada...

In [74]:
extrair_produtos_pagina()

[('iPhone 11 Apple 64GB Branco 6,1” 12MP iOS',
  3719.07,
  4,
  58,
  'https://www.magazineluiza.com.br/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/'),
 ('iPhone 11 Apple 64GB Roxo 6,1” 12MP iOS',
  3719.07,
  5,
  1,
  'https://www.magazineluiza.com.br/iphone-11-apple-64gb-roxo-61-12mp-ios/p/155610900/te/ip11/'),
 ('Smartphone Samsung Galaxy S20 FE 128GB Cloud Mint',
  2099.0,
  4,
  48,
  'https://www.magazineluiza.com.br/smartphone-samsung-galaxy-s20-fe-128gb-cloud-mint-4g-6gb-ram-tela-65-cam-tripla-selfie-32mp/p/155630400/te/gs2f/'),
 ('iPhone 11 Apple 64GB Preto 6,1” 12MP iOS',
  3719.07,
  4,
  80,
  'https://www.magazineluiza.com.br/iphone-11-apple-64gb-preto-61-12mp-ios/p/155610500/te/ip11/'),
 ('Smartphone Samsung Galaxy S20 FE 128GB Cloud White',
  2099.0,
  4,
  51,
  'https://www.magazineluiza.com.br/smartphone-samsung-galaxy-s20-fe-128gb-cloud-white-4g-6gb-ram-tela-65-cam-tripla-selfie-32mp/p/155630000/te/gs2f/'),
 ('iPhone XR Apple 64GB Preto 6,1” 12MP iO

Não acredito que tudo até agora rodou liso feito manteiga 🧈